In [1]:
import os

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import anndata
import scanpy as sc
sc.settings.n_jobs = 50
sc.settings.set_figure_params(dpi=300, dpi_save=300, frameon=False, figsize=(4, 4), fontsize=8, facecolor='white')

In [2]:
import scipy.spatial
import scipy.stats
import statsmodels.stats.multitest

def get_close_contact_cell_pairs_cis(query_df, contact_radius, slice_col='slice_id', 
                                 xy_cols=['center_x', 'center_y']):
    '''Get pairs of cells that are within a given contact radius'''
    # Get the coordinates of the query cells 
    query_coords = np.array(query_df[xy_cols])
    
    # Get the contact cells slice-by-slice
    slice_ids = np.unique(query_df[slice_col])
    contact_cell_pairs = []
    
    for slice_id in slice_ids:
        slice_mask_q = query_df[slice_col] == slice_id
        
        query_coords_slice = query_coords[slice_mask_q]
        query_cell_id_slice = query_df.index[slice_mask_q]
        
        # Skip if there is no cell in this slice
        if len(query_coords_slice) == 0:
            continue
            
        # Get the cell pairs within the contact radius
        point_tree = scipy.spatial.cKDTree(query_coords_slice)
        neighbor_ids = point_tree.query_ball_point(query_coords_slice, contact_radius)
        
        for i in range(len(neighbor_ids)):
            for j in neighbor_ids[i]:
                if query_cell_id_slice[i] < query_cell_id_slice[j]:
                    contact_cell_pairs.append((query_cell_id_slice[i], query_cell_id_slice[j]))
                    
    return np.array(contact_cell_pairs)

def random_select_non_contact_cell_pairs_cis(query_df, contact_cell_pairs):
    '''Random select non-contact cell pairs. The number of selected non-contact pairs are
    the same as the number of the contact pairs.'''
    non_contact_query_cells = list(query_df.index[~query_df.index.isin(
                                   list(contact_cell_pairs[:, 0]) + list(contact_cell_pairs[:, 1]))])
    np.random.shuffle(non_contact_query_cells)
    N_ncs = len(non_contact_query_cells)
    N_max_pairs = int(N_ncs * (N_ncs - 1) / 2)
    N_pairs = len(contact_cell_pairs)
    N_pairs = min(N_pairs, N_max_pairs)
    
    if N_pairs == 0:
        return np.array([])
    
    return np.stack((np.random.choice(non_contact_query_cells[:N_ncs//2], N_pairs),
                     np.random.choice(non_contact_query_cells[N_ncs//2:], N_pairs)), axis=1)
    

def get_close_contact_cell_pairs(query_df, ref_df, contact_radius, slice_col='slice_id', 
                                 xy_cols=['center_x', 'center_y']):
    '''Get pairs of cells that are within a given contact radius'''
    
    # Get the coordinates of the query cells and reference cells
    query_coords = np.array(query_df[xy_cols])
    ref_coords = np.array(ref_df[xy_cols])
    
    # Get the contact cells to reference cells slice-by-slice
    slice_ids = np.unique(query_df[slice_col])
    contact_cell_pairs = []
    
    for slice_id in slice_ids:
        slice_mask_q = query_df[slice_col] == slice_id
        slice_mask_r = ref_df[slice_col] == slice_id
        
        query_coords_slice = query_coords[slice_mask_q]
        ref_coords_slice = ref_coords[slice_mask_r]
        
        query_cell_id_slice = query_df.index[slice_mask_q]
        ref_cell_id_slice = ref_df.index[slice_mask_r]
    
        # Skip if there is no cell in this slice
        if (len(query_coords_slice) == 0) or (len(ref_coords_slice) == 0):
            continue
            
        # Get the cell pairs within the contact radius
        point_tree = scipy.spatial.cKDTree(ref_coords_slice)
        neighbor_ids = point_tree.query_ball_point(query_coords_slice, contact_radius)
        
        for i in range(len(neighbor_ids)):
            for j in neighbor_ids[i]:
                contact_cell_pairs.append((query_cell_id_slice[i], ref_cell_id_slice[j]))
        
    return np.array(contact_cell_pairs)

def random_select_non_contact_cell_pairs(query_df, ref_df, contact_cell_pairs):
    '''Random select non-contact cell pairs. The number of selected non-contact pairs are
    the same as the number of the contact pairs.'''
    non_contact_query_cells = query_df.index[~query_df.index.isin(contact_cell_pairs[:, 0])]
    non_contact_ref_cells = ref_df.index[~ref_df.index.isin(contact_cell_pairs[:, 1])]
    
    N_max_pairs = len(non_contact_query_cells) * len(non_contact_ref_cells)
    N_pairs = len(contact_cell_pairs)
    N_pairs = min(N_pairs, N_max_pairs)
    
    if N_pairs == 0:
        return np.array([])
    
    return np.stack((np.random.choice(non_contact_query_cells, N_pairs),
                     np.random.choice(non_contact_ref_cells, N_pairs)), axis=1)

def random_select_cell_pairs(query_df, ref_df, contact_cell_pairs):
    '''Random select cell pairs. The number of selected pairs are
    the same as the number of the contact pairs.'''
    N_pairs = len(contact_cell_pairs)
    
    return np.stack((np.random.choice(query_df.index, N_pairs),
                     np.random.choice(ref_df.index, N_pairs)), axis=1)

def get_expressions_for_cells(adata, cell_names):
    '''Get the expressions for given cells.'''
    # Get the indices of cells
    cell_id_map = pd.DataFrame({'cell_name': list(adata.obs.index), 
                                 'index': list(range(adata.shape[0]))}).set_index('cell_name')
    cell_ids = np.array(cell_id_map['index'].loc[cell_names])
    return adata.X[cell_ids]
    

def get_lr_pair_expressions_for_cell_pairs(adata_ligand_cells, adata_receptor_cells, cell_pairs,
                                          ligand_gene, receptor_gene):
    '''Get the L-R pair expressions for a given set of cell pairs.'''
    
    # Get the indices of cells
    ligand_cell_id_map = pd.DataFrame({'cell_id': list(adata_ligand_cells.obs.index), 
                                 'index': list(range(adata_ligand_cells.shape[0]))}).set_index('cell_id')
    receptor_cell_id_map = pd.DataFrame({'cell_id': list(adata_receptor_cells.obs.index), 
                                 'index': list(range(adata_receptor_cells.shape[0]))}).set_index('cell_id')
    
    lig_cell_ids = np.array(ligand_cell_id_map['index'].loc[cell_pairs[:, 0]])
    rec_cell_ids = np.array(receptor_cell_id_map['index'].loc[cell_pairs[:, 1]])
    
    # Get the indices of genes
    lig_gene_id = np.where(adata_ligand_cells.var.index == ligand_gene)[0][0]
    rec_gene_id = np.where(adata_receptor_cells.var.index == receptor_gene)[0][0]
    
    # Get the expressions
    lig_exp = adata_ligand_cells.X[lig_cell_ids, lig_gene_id]
    rec_exp = adata_receptor_cells.X[rec_cell_ids, rec_gene_id]
    
    return lig_exp * rec_exp

In [3]:
# Load the imputed lr expressions
adata_ligand = sc.read_h5ad('../imputed_lr_exps/adate_imputed_lr_pair_ligand_expression.h5ad')
adata_receptor = sc.read_h5ad('../imputed_lr_exps/adate_imputed_lr_pair_receptor_expression.h5ad')

# Define the L-R pairs
lr_pairs = list(adata_ligand.var.index)
assert(np.all(np.array(adata_ligand.var.index) == np.array(adata_receptor.var.index)))

# Find out which L-R pairs are symmetric
symmetric_lr_pairs = []
for lr_pair in lr_pairs:
    lrps = lr_pair.split('_')
    if (len(lrps) == 2) and (lrps[0] == lrps[1]):
        symmetric_lr_pairs.append(lr_pair)

In [4]:
%%time
output_path = 'lr_interactions'
os.makedirs(output_path, exist_ok=True)
cell_by_region_path = '/home/xingjiepan/data/whole_brain/analysis/20230808_cell_cell_contacts/cells_by_regions'
contact_path = '/home/xingjiepan/data/whole_brain/analysis/20230808_cell_cell_contacts/outputs_30um'


major_brain_regions = ['Cerebellum', 'Cortical_subplate', 'Fiber_tracts', 'Hippocampus',
       'Isocortex', 'Medulla', 'Olfactory', 'Pallidum', 'Pons', 'Striatum',
       'Thalamus', 'Ventricular_systems', 'anterior_HY', 'anterior_MB',
       'posterior_HY', 'posterior_MB']
contact_radius = 30


for region in major_brain_regions:
    
    print(region)
    lr_interaction_dict = {'subclass_ligand': [], 'subclass_receptor': [], 'LR_pair': [], 'N_contact_pairs':[],
                       'mean_contact_lr_exp': [], 'exp_fraction': [], 'fold_change': [], 'pval': []}
    
    # Only keep the cells in the region
    region_df = pd.read_csv(os.path.join(cell_by_region_path, f'{region}.csv'), index_col=0)
    cells_in_region = list(region_df.index)
    adata_ligand_region = adata_ligand[adata_ligand.obs.index.isin(cells_in_region)]
    adata_receptor_region = adata_receptor[adata_receptor.obs.index.isin(cells_in_region)]
    adata_ligand_region.obs['slice_id'] = region_df['slice_id']
    adata_ligand_region.obs['center_x'] = region_df['center_x']
    adata_ligand_region.obs['center_y'] = region_df['center_y']
    adata_receptor_region.obs['slice_id'] = region_df['slice_id']
    adata_receptor_region.obs['center_x'] = region_df['center_x']
    adata_receptor_region.obs['center_y'] = region_df['center_y']
    
    # Get the contact pairs
    close_contact_subclass_df = pd.read_csv(os.path.join(contact_path, f'{region}_close_contacts.csv'))

    # Calculate the L-R pair enrichment in close-contact cells
    for i in range(close_contact_subclass_df.shape[0]):
        ct1 = close_contact_subclass_df['cell_type1'][i]
        ct2 = close_contact_subclass_df['cell_type2'][i]
            
        print(ct1, ct2)
    
        if ct1 == ct2:
            ct_lr_pairs = [(ct1, ct2)]
        else:
            ct_lr_pairs = [(ct1, ct2), (ct2, ct1)]
    
        # The two cell types can either be ligand or receptors
        for ct_l, ct_r in ct_lr_pairs:
    
            adata_ligand_cells = adata_ligand_region[adata_ligand_region.obs['subclass_label_transfer'] == ct_l]
            adata_receptor_cells = adata_receptor_region[
                                            adata_receptor_region.obs['subclass_label_transfer'] == ct_r]
            
            # Get the contact cell type pairs
            if ct1 == ct2:
                contact_cell_pairs = get_close_contact_cell_pairs_cis(adata_ligand_cells.obs, contact_radius)
            else:
                contact_cell_pairs = get_close_contact_cell_pairs(adata_ligand_cells.obs, adata_receptor_cells.obs, 
                                                      contact_radius)
        
            # Get non-contact cell type pairs
            if ct1 == ct2:
                noncontact_cell_pairs = random_select_non_contact_cell_pairs_cis(adata_ligand_cells.obs, 
                                                             contact_cell_pairs)
            else:
                noncontact_cell_pairs = random_select_non_contact_cell_pairs(adata_ligand_cells.obs, 
                                                             adata_receptor_cells.obs, contact_cell_pairs)
                
            print('N_contact_pairs =', contact_cell_pairs.shape[0])
            print('N_noncontact_cell_pairs =', noncontact_cell_pairs.shape[0])
            if (contact_cell_pairs.shape[0] < 10) or (noncontact_cell_pairs.shape[0] < 10):
                continue
               
            # Get the expressions of contact and non-contact pairs of cells
            contact_lig_exps = get_expressions_for_cells(adata_ligand_cells, contact_cell_pairs[:, 0])
            contact_rec_exps = get_expressions_for_cells(adata_receptor_cells, contact_cell_pairs[:, 1])
            contact_lr_exps = contact_lig_exps * contact_rec_exps
            contact_lr_exps = np.log(1 + contact_lr_exps)
            
            noncontact_lig_exps = get_expressions_for_cells(adata_ligand_cells, 
                                                               noncontact_cell_pairs[:, 0])
            noncontact_rec_exps = get_expressions_for_cells(adata_receptor_cells, 
                                                               noncontact_cell_pairs[:, 1])
            noncontact_lr_exps = noncontact_lig_exps * noncontact_rec_exps
            noncontact_lr_exps = np.log(1 + noncontact_lr_exps)
            
            mean_contact_exps = np.mean(contact_lr_exps, axis=0)
            exp_fractions = np.sum(contact_lr_exps > 0, axis=0) / contact_lr_exps.shape[0]
            fold_changes = np.mean(contact_lr_exps, axis=0) / np.mean(noncontact_lr_exps, axis=0)
            pvals = scipy.stats.ttest_ind(contact_lr_exps, noncontact_lr_exps, 
                                             equal_var=False, axis=0, alternative='greater')[1]
    
            for i, lr_pair in enumerate(lr_pairs):
                # Do not double count the symmetric LR pairs
                if (lr_pair in symmetric_lr_pairs) and (ct_l == ct2):
                    continue

                # Record the results
                lr_interaction_dict['subclass_ligand'].append(ct_l)
                lr_interaction_dict['subclass_receptor'].append(ct_r)
                lr_interaction_dict['LR_pair'].append(lr_pair)
                lr_interaction_dict['N_contact_pairs'].append(len(contact_cell_pairs))
                lr_interaction_dict['mean_contact_lr_exp'].append(mean_contact_exps[i])
                lr_interaction_dict['exp_fraction'].append(exp_fractions[i])
                lr_interaction_dict['fold_change'].append(fold_changes[i])
                lr_interaction_dict['pval'].append(pvals[i])

    lr_interaction_df = pd.DataFrame(lr_interaction_dict)
    lr_interaction_df = lr_interaction_df[~np.isnan(lr_interaction_df['pval'])]

    lr_interaction_df['pval_adjusted'] = statsmodels.stats.multitest.multipletests(
                                            lr_interaction_df['pval'], method='fdr_bh')[1]
    lr_interaction_df.to_csv(os.path.join(output_path, f'{region}_LR_interactions_before_filter.csv'))
    
    
    mask = lr_interaction_df['pval_adjusted'] < 0.01
    mask = mask & (lr_interaction_df['fold_change'] > 2)
    mask = mask & (lr_interaction_df['exp_fraction'] > 0.4)

    lr_interaction_df_filtered = lr_interaction_df[mask].sort_values(['pval_adjusted', 'fold_change'], 
                                                                 ascending=[True, False])
    lr_interaction_df_filtered.to_csv(os.path.join(output_path, f'{region}_LR_interactions.csv'))
    

Cerebellum


<timed exec>:25: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
<timed exec>:28: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.


CB Granule Glut CB Granule Glut
N_contact_pairs = 2053926
N_noncontact_cell_pairs = 2053926


<timed exec>:89: RuntimeWarning: divide by zero encountered in divide
<timed exec>:89: RuntimeWarning: invalid value encountered in divide


Bergmann NN Bergmann NN
N_contact_pairs = 42393
N_noncontact_cell_pairs = 42393


...

PAG Pou4f2 Glut PAG Pou4f2 Glut
N_contact_pairs = 558
N_noncontact_cell_pairs = 558
MRN-VTN-PPN Pax5 Cdh23 Gaba PAG-MRN Pou3f1 Glut
N_contact_pairs = 62
N_noncontact_cell_pairs = 62
N_contact_pairs = 62
N_noncontact_cell_pairs = 62
CS-PRNr-DR En1 Sox2 Gaba Endo NN
N_contact_pairs = 183
N_noncontact_cell_pairs = 183


<timed exec>:89: RuntimeWarning: divide by zero encountered in divide
<timed exec>:89: RuntimeWarning: invalid value encountered in divide
<timed exec>:89: RuntimeWarning: divide by zero encountered in divide
<timed exec>:89: RuntimeWarning: invalid value encountered in divide
<timed exec>:89: RuntimeWarning: divide by zero encountered in divide
<timed exec>:89: RuntimeWarning: invalid value encountered in divide
<timed exec>:89: RuntimeWarning: divide by zero encountered in divide
<timed exec>:89: RuntimeWarning: invalid value encountered in divide


N_contact_pairs = 183
N_noncontact_cell_pairs = 183
SC Tnnt1 Gli3 Gaba SCig Tfap2b Chrnb3 Glut
N_contact_pairs = 97
N_noncontact_cell_pairs = 97
N_contact_pairs = 97
N_noncontact_cell_pairs = 97


<timed exec>:89: RuntimeWarning: divide by zero encountered in divide
<timed exec>:89: RuntimeWarning: invalid value encountered in divide
<timed exec>:89: RuntimeWarning: divide by zero encountered in divide
<timed exec>:89: RuntimeWarning: invalid value encountered in divide
<timed exec>:89: RuntimeWarning: divide by zero encountered in divide
<timed exec>:89: RuntimeWarning: invalid value encountered in divide


CPU times: user 18min 49s, sys: 4min 16s, total: 23min 6s
Wall time: 23min 7s


In [5]:
major_brain_regions = ['Cerebellum', 'Cortical_subplate', 'Fiber_tracts', 'Hippocampus',
       'Isocortex', 'Medulla', 'Olfactory', 'Pallidum', 'Pons', 'Striatum',
       'Thalamus', 'Ventricular_systems', 'anterior_HY', 'anterior_MB',
       'posterior_HY', 'posterior_MB']

all_lr_df_list = [pd.read_csv(os.path.join('lr_interactions', f'{region}_LR_interactions.csv'), index_col=0)
              for region in major_brain_regions]
all_lr_df = pd.concat(all_lr_df_list)

In [6]:
import json
allowed_pairs_by_regions = {}
for r in major_brain_regions:
    df = pd.read_csv(os.path.join('lr_interactions', f'{r}_LR_interactions.csv'), index_col=0)
    df = df[df['pval_adjusted'] < 0.01]
    allowed_pairs_by_regions[r] = list(set([tuple(sorted(p)) 
                                            for p in df[['subclass_ligand', 'subclass_receptor']].values]))
    
with open('allowed_pairs_by_regions.json', 'w') as f:
    json.dump(allowed_pairs_by_regions, f)

In [20]:
#ct1, ct2 = 'BAM NN', 'Endo NN'
#ct1, ct2 = 'Oligo NN', 'Microglia NN'
#ct1, ct2 = 'Astro-OLF NN', 'MOB-STR-CTX Inh IMN'
#ct1, ct2 = 'Astro-TE NN', 'MOB-STR-CTX Inh IMN'
#ct1, ct2 = 'OB Meis2 Gaba', 'OB-out Frmd7 Gaba'
#ct1, ct2 = 'CHOR NN', 'Astro-Epen NN'
#ct1, ct2 = 'SCs Dmbx1 Gaba', 'SCig Foxb1 Glut'
#ct1, ct2 = 'SCs Dmbx1 Gaba', 'SCs Lef1 Gli3 Gaba'
#ct1, ct2 = 'Endo NN', 'SMC NN'
#ct1, ct2 = 'Astro-NT NN', 'PG-TRN-LRN Fat2 Glut'
#ct1, ct2 = 'Astro-Epen NN', 'HPF CR Glut'
#ct1, ct2 = 'Microglia NN', 'Oligo NN'


#all_lr_df[((all_lr_df['subclass_ligand'] == ct1) & (all_lr_df['subclass_receptor'] == ct2))
#         |((all_lr_df['subclass_ligand'] == ct2) & (all_lr_df['subclass_receptor'] == ct1))]

all_lr_df[(all_lr_df['subclass_ligand'] == ct1) & (all_lr_df['subclass_receptor'] == ct2)]

,subclass_ligand,subclass_receptor,LR_pair,N_contact_pairs,mean_contact_lr_exp,exp_fraction,fold_change,pval,pval_adjusted
76422,Oligo NN,Microglia NN,PSAP_GPR37,3355,0.093448,0.414903,3.454168,0.000000e+00,0.000000e+00
76967,Oligo NN,Microglia NN,CNTN2_CNTN2_CNTNAP2,3355,0.092627,0.564829,2.180525,2.492733e-37,9.778891e-36
77251,Oligo NN,Microglia NN,ITGA4_ITGB1_VCAM1,3355,0.053163,0.752310,2.158876,1.203294e-25,3.709455e-24
76342,Oligo NN,Microglia NN,SEMA3C_NRP1_PLXNA2,3355,0.003433,0.433681,2.048251,6.849769e-09,1.100999e-07
76346,Oligo NN,Microglia NN,SEMA3D_NRP1_PLXNA2,3355,0.006422,0.422355,2.107368,7.881533e-05,8.261768e-04
60550,Oligo NN,Microglia NN,PSAP_GPR37,38973,0.113455,0.430811,2.195534,0.000000e+00,0.000000e+00
126570,Oligo NN,Microglia NN,EFNB3_EPHB1,938,0.234016,0.551173,2.206548,1.417882e-17,5.275675e-16
126771,Oligo NN,Microglia NN,SEMA4D_PLXNB3,938,0.142110,0.641791,2.083896,1.521596e-13,4.736404e-12
125967,Oligo NN,Microglia NN,PSAP_GPR37,938,0.118143,0.447761,2.299828,7.452641e-13,2.233656e-11
126780,Oligo NN,Microglia NN,SEMA5A_PLXNB3,938,0.044093,0.609808,2.627091,2.444609e-04,3.196736e-03


In [8]:
pair_set = set([tuple(sorted(p)) for p in all_lr_df[['subclass_ligand', 'subclass_receptor']].values])
len(pair_set)

651

In [9]:
pair_set

{('ABC NN', 'ABC NN'),
 ('ABC NN', 'Astro-TE NN'),
 ('ABC NN', 'BAM NN'),
 ('ABC NN', 'Endo NN'),
 ('ABC NN', 'Ependymal NN'),
 ('ABC NN', 'Lymphoid NN'),
 ('ABC NN', 'Microglia NN'),
 ('ABC NN', 'OEC NN'),
 ('ABC NN', 'Peri NN'),
 ('ABC NN', 'SMC NN'),
 ('ABC NN', 'VLMC NN'),
 ('ACB-BST-FS D1 Gaba', 'ACB-BST-FS D1 Gaba'),
 ('ACB-BST-FS D1 Gaba', 'CEA-AAA-BST Six3 Sp9 Gaba'),
 ('ACB-BST-FS D1 Gaba', 'Microglia NN'),
 ('AD Serpinb7 Glut', 'AD Serpinb7 Glut'),
 ('AD Serpinb7 Glut', 'Astro-NT NN'),
 ('AD Serpinb7 Glut', 'Endo NN'),
 ('ADP-MPO Trp73 Glut', 'ADP-MPO Trp73 Glut'),
 ('AHN Onecut3 Gaba', 'AHN Onecut3 Gaba'),
 ('AHN-RCH-LHA Otp Fezf1 Glut', 'AHN-RCH-LHA Otp Fezf1 Glut'),
 ('AHN-SBPV-PVHd Pdrm12 Gaba', 'AHN-SBPV-PVHd Pdrm12 Gaba'),
 ('APN C1ql4 Glut', 'APN C1ql4 Glut'),
 ('ARH-PVp Tbx3 Gaba', 'ARH-PVp Tbx3 Gaba'),
 ('ARH-PVp Tbx3 Gaba', 'Endo NN'),
 ('ARH-PVp Tbx3 Gaba', 'TU-ARH Otp Six6 Gaba'),
 ('ARH-PVp Tbx3 Glut', 'ARH-PVp Tbx3 Glut'),
 ('ARH-PVp Tbx3 Glut', 'DMH-LHA Gsx1 Ga

In [10]:
pair_set_15um = {('ABC NN', 'ABC NN'),
 ('ABC NN', 'Astro-TE NN'),
 ('ABC NN', 'BAM NN'),
 ('ABC NN', 'Endo NN'),
 ('ABC NN', 'OEC NN'),
 ('ABC NN', 'Peri NN'),
 ('ABC NN', 'SMC NN'),
 ('ABC NN', 'VLMC NN'),
 ('ACB-BST-FS D1 Gaba', 'ACB-BST-FS D1 Gaba'),
 ('AD Serpinb7 Glut', 'AD Serpinb7 Glut'),
 ('ADP-MPO Trp73 Glut', 'ADP-MPO Trp73 Glut'),
 ('AHN Onecut3 Gaba', 'AHN Onecut3 Gaba'),
 ('APN C1ql2 Glut', 'APN C1ql2 Glut'),
 ('APN C1ql4 Glut', 'APN C1ql4 Glut'),
 ('ARH-PVp Tbx3 Gaba', 'ARH-PVp Tbx3 Gaba'),
 ('ARH-PVp Tbx3 Glut', 'ARH-PVp Tbx3 Glut'),
 ('AVPV-MEPO-SFO Tbr1 Glut', 'AVPV-MEPO-SFO Tbr1 Glut'),
 ('AVPV-MEPO-SFO Tbr1 Glut', 'Tanycyte NN'),
 ('Astro-CB NN', 'Astro-CB NN'),
 ('Astro-CB NN', 'Bergmann NN'),
 ('Astro-CB NN', 'CB Granule Glut'),
 ('Astro-CB NN', 'Endo NN'),
 ('Astro-CB NN', 'Microglia NN'),
 ('Astro-CB NN', 'Oligo NN'),
 ('Astro-NT NN', 'Astro-NT NN'),
 ('Astro-NT NN', 'BAM NN'),
 ('Astro-NT NN', 'Endo NN'),
 ('Astro-NT NN', 'Ependymal NN'),
 ('Astro-NT NN', 'Microglia NN'),
 ('Astro-NT NN', 'OPC NN'),
 ('Astro-NT NN', 'Oligo NN'),
 ('Astro-NT NN', 'Peri NN'),
 ('Astro-NT NN', 'SCsg Gabrr2 Gaba'),
 ('Astro-NT NN', 'SMC NN'),
 ('Astro-NT NN', 'TRS-BAC Sln Glut'),
 ('Astro-NT NN', 'VLMC NN'),
 ('Astro-OLF NN', 'Astro-OLF NN'),
 ('Astro-OLF NN', 'Endo NN'),
 ('Astro-OLF NN', 'Microglia NN'),
 ('Astro-OLF NN', 'OB Dopa-Gaba'),
 ('Astro-OLF NN', 'OB Eomes Ms4a15 Glut'),
 ('Astro-OLF NN', 'OB Meis2 Thsd7b Gaba'),
 ('Astro-OLF NN', 'OB-STR-CTX Inh IMN'),
 ('Astro-OLF NN', 'OB-out Frmd7 Gaba'),
 ('Astro-OLF NN', 'Oligo NN'),
 ('Astro-OLF NN', 'Peri NN'),
 ('Astro-OLF NN', 'SMC NN'),
 ('Astro-TE NN', 'Astro-TE NN'),
 ('Astro-TE NN', 'BAM NN'),
 ('Astro-TE NN', 'DG-PIR Ex IMN'),
 ('Astro-TE NN', 'Endo NN'),
 ('Astro-TE NN', 'Ependymal NN'),
 ('Astro-TE NN', 'HPF CR Glut'),
 ('Astro-TE NN', 'Microglia NN'),
 ('Astro-TE NN', 'OB-STR-CTX Inh IMN'),
 ('Astro-TE NN', 'OPC NN'),
 ('Astro-TE NN', 'Oligo NN'),
 ('Astro-TE NN', 'Peri NN'),
 ('Astro-TE NN', 'SMC NN'),
 ('Astro-TE NN', 'VLMC NN'),
 ('Astroependymal NN', 'Astroependymal NN'),
 ('Astroependymal NN', 'Ependymal NN'),
 ('Astroependymal NN', 'MH Tac2 Glut'),
 ('Astroependymal NN', 'VLMC NN'),
 ('BAM NN', 'BAM NN'),
 ('BAM NN', 'Bergmann NN'),
 ('BAM NN', 'CHOR NN'),
 ('BAM NN', 'Endo NN'),
 ('BAM NN', 'Microglia NN'),
 ('BAM NN', 'OEC NN'),
 ('BAM NN', 'Peri NN'),
 ('BAM NN', 'SMC NN'),
 ('BAM NN', 'VLMC NN'),
 ('BST Tac2 Gaba', 'BST Tac2 Gaba'),
 ('BST-MPN Six3 Nrgn Gaba', 'BST-MPN Six3 Nrgn Gaba'),
 ('BST-po Iigp1 Glut', 'BST-po Iigp1 Glut'),
 ('Bergmann NN', 'Bergmann NN'),
 ('Bergmann NN', 'CB PLI Gly-Gaba'),
 ('Bergmann NN', 'CBX Golgi Gly-Gaba'),
 ('Bergmann NN', 'CBX Purkinje Gaba'),
 ('Bergmann NN', 'Endo NN'),
 ('Bergmann NN', 'Oligo NN'),
 ('Bergmann NN', 'Peri NN'),
 ('CA1-ProS Glut', 'CA1-ProS Glut'),
 ('CA1-ProS Glut', 'Pvalb chandelier Gaba'),
 ('CA2-FC-IG Glut', 'CA2-FC-IG Glut'),
 ('CA3 Glut', 'CA3 Glut'),
 ('CB Granule Glut', 'CB Granule Glut'),
 ('CB Granule Glut', 'CB PLI Gly-Gaba'),
 ('CB Granule Glut', 'CBX Golgi Gly-Gaba'),
 ('CB Granule Glut', 'DCO UBC Glut'),
 ('CB PLI Gly-Gaba', 'CB PLI Gly-Gaba'),
 ('CB PLI Gly-Gaba', 'CBX Purkinje Gaba'),
 ('CBN Dmbx1 Gaba', 'CBN Dmbx1 Gaba'),
 ('CBN Dmbx1 Gaba', 'Oligo NN'),
 ('CBX MLI Cdh22 Gaba', 'CBX MLI Cdh22 Gaba'),
 ('CBX MLI Cdh22 Gaba', 'CBX MLI Megf11 Gaba'),
 ('CEA-AAA-BST Six3 Sp9 Gaba', 'CEA-AAA-BST Six3 Sp9 Gaba'),
 ('CEA-BST Ebf1 Pdyn Gaba', 'CEA-BST Ebf1 Pdyn Gaba'),
 ('CHOR NN', 'CHOR NN'),
 ('CHOR NN', 'Endo NN'),
 ('CHOR NN', 'Lymphoid NN'),
 ('CHOR NN', 'Peri NN'),
 ('CHOR NN', 'VLMC NN'),
 ('CLA-EPd-CTX Car3 Glut', 'CLA-EPd-CTX Car3 Glut'),
 ('CLA-EPd-CTX Car3 Glut', 'L2/3 IT RSP Glut'),
 ('CM-IAD-CL-PCN Sema5b Glut', 'CM-IAD-CL-PCN Sema5b Glut'),
 ('COAa-PAA-MEA Barhl2 Glut', 'COAa-PAA-MEA Barhl2 Glut'),
 ('COAa-PAA-MEA Barhl2 Glut', 'Oligo NN'),
 ('COAp Grxcr2 Glut', 'COAp Grxcr2 Glut'),
 ('CS-PRNr-PCG Tmem163 Otp Gaba', 'CS-PRNr-PCG Tmem163 Otp Gaba'),
 ('CS-RPO Meis2 Gaba', 'CS-RPO Meis2 Gaba'),
 ('CT SUB Glut', 'CT SUB Glut'),
 ('CT SUB Glut', 'VLMC NN'),
 ('CUN-PPN Evx2 Meis2 Glut', 'CUN-PPN Evx2 Meis2 Glut'),
 ('DC NN', 'Endo NN'),
 ('DC NN', 'VLMC NN'),
 ('DCO UBC Glut', 'DCO UBC Glut'),
 ('DG Glut', 'DG Glut'),
 ('DG-PIR Ex IMN', 'DG-PIR Ex IMN'),
 ('DG-PIR Ex IMN', 'Endo NN'),
 ('DG-PIR Ex IMN', 'Lamp5 Lhx6 Gaba'),
 ('DG-PIR Ex IMN', 'Microglia NN'),
 ('DG-PIR Ex IMN', 'Oligo NN'),
 ('DMH Hmx2 Gaba', 'DMH Hmx2 Gaba'),
 ('DMH Hmx2 Glut', 'DMH Hmx2 Glut'),
 ('DMH Nkx2-4 Glut', 'DMH Nkx2-4 Glut'),
 ('DMH-LHA Gsx1 Gaba', 'DMH-LHA Gsx1 Gaba'),
 ('DTN-LDT-IPN Otp Pax3 Gaba', 'DTN-LDT-IPN Otp Pax3 Gaba'),
 ('ENTmv-PA-COAp Glut', 'ENTmv-PA-COAp Glut'),
 ('Endo NN', 'Endo NN'),
 ('Endo NN', 'Lymphoid NN'),
 ('Endo NN', 'MH Tac2 Glut'),
 ('Endo NN', 'Microglia NN'),
 ('Endo NN', 'OB-STR-CTX Inh IMN'),
 ('Endo NN', 'OB-out Frmd7 Gaba'),
 ('Endo NN', 'OEC NN'),
 ('Endo NN', 'OPC NN'),
 ('Endo NN', 'Oligo NN'),
 ('Endo NN', 'PVH-SO-PVa Otp Glut'),
 ('Endo NN', 'Peri NN'),
 ('Endo NN', 'SMC NN'),
 ('Endo NN', 'TRS-BAC Sln Glut'),
 ('Endo NN', 'VLMC NN'),
 ('Ependymal NN', 'Ependymal NN'),
 ('Ependymal NN', 'MEA-BST Sox6 Gaba'),
 ('Ependymal NN', 'Microglia NN'),
 ('Ependymal NN', 'OB-STR-CTX Inh IMN'),
 ('Ependymal NN', 'Tanycyte NN'),
 ('GPe-SI Sox6 Cyp26b1 Gaba', 'GPe-SI Sox6 Cyp26b1 Gaba'),
 ('GPe-SI Sox6 Cyp26b1 Gaba', 'Oligo NN'),
 ('HB Calcb Chol', 'HB Calcb Chol'),
 ('HPF CR Glut', 'HPF CR Glut'),
 ('Hypendymal NN', 'Hypendymal NN'),
 ('IC Six3 En2 Gaba', 'IC Six3 En2 Gaba'),
 ('IF-RL-CLI-PAG Foxa1 Glut', 'IF-RL-CLI-PAG Foxa1 Glut'),
 ('IPN Otp Crisp1 Gaba', 'IPN Otp Crisp1 Gaba'),
 ('IPN-LDT Vsx2 Nkx6-1 Glut', 'IPN-LDT Vsx2 Nkx6-1 Glut'),
 ('IT AON-TT-DP Glut', 'IT AON-TT-DP Glut'),
 ('IT EP-CLA Glut', 'IT EP-CLA Glut'),
 ('L2 IT PPP-APr Glut', 'L2 IT PPP-APr Glut'),
 ('L2/3 IT CTX Glut', 'L2/3 IT CTX Glut'),
 ('L2/3 IT CTX Glut', 'L2/3 IT RSP Glut'),
 ('L2/3 IT CTX Glut', 'Vip Gaba'),
 ('L2/3 IT PIR-ENTl Glut', 'L2/3 IT PIR-ENTl Glut'),
 ('L2/3 IT PPP Glut', 'L2/3 IT PPP Glut'),
 ('L2/3 IT RSP Glut', 'L2/3 IT RSP Glut'),
 ('L2/3 IT RSP Glut', 'Vip Gaba'),
 ('L5 IT CTX Glut', 'L5 IT CTX Glut'),
 ('L5 NP CTX Glut', 'L5 NP CTX Glut'),
 ('L5 PPP Glut', 'L5 PPP Glut'),
 ('L5 PPP Glut', 'NP PPP Glut'),
 ('L5/6 IT TPE-ENT Glut', 'L5/6 IT TPE-ENT Glut'),
 ('L6 CT CTX Glut', 'L6 CT CTX Glut'),
 ('L6 IT CTX Glut', 'L6 IT CTX Glut'),
 ('L6b CTX Glut', 'L6b CTX Glut'),
 ('L6b/CT ENT Glut', 'L6b/CT ENT Glut'),
 ('LDT-PCG St18 Gaba', 'LDT-PCG St18 Gaba'),
 ('LDT-PCG Vsx2 Lhx4 Glut', 'LDT-PCG Vsx2 Lhx4 Glut'),
 ('LDT-PCG-CS Gata3 Lhx1 Gaba', 'LDT-PCG-CS Gata3 Lhx1 Gaba'),
 ('LGv-SPFp-SPFm Nkx2-2 Tcf7l2 Gaba', 'LGv-SPFp-SPFm Nkx2-2 Tcf7l2 Gaba'),
 ('LH Pou4f1 Sox1 Glut', 'LH Pou4f1 Sox1 Glut'),
 ('LHA-AHN-PVH Otp Trh Glut', 'LHA-AHN-PVH Otp Trh Glut'),
 ('LSX Nkx2-1 Gaba', 'LSX Nkx2-1 Gaba'),
 ('LSX Otx2 Gaba', 'LSX Otx2 Gaba'),
 ('LSX Prdm12 Zeb2 Gaba', 'LSX Prdm12 Zeb2 Gaba'),
 ('LSX Sall3 Lmo1 Gaba', 'LSX Sall3 Lmo1 Gaba'),
 ('LSX Sall3 Pax6 Gaba', 'LSX Sall3 Pax6 Gaba'),
 ('Lamp5 Gaba', 'Lamp5 Gaba'),
 ('Lamp5 Lhx6 Gaba', 'Lamp5 Lhx6 Gaba'),
 ('Lymphoid NN', 'Lymphoid NN'),
 ('Lymphoid NN', 'OEC NN'),
 ('Lymphoid NN', 'Peri NN'),
 ('Lymphoid NN', 'VLMC NN'),
 ('MB-MY Tph2 Glut-Sero', 'MB-MY Tph2 Glut-Sero'),
 ('MEA-BST Lhx6 Nfib Gaba', 'MEA-BST Lhx6 Nfib Gaba'),
 ('MEA-BST Lhx6 Sp9 Gaba', 'MEA-BST Lhx6 Sp9 Gaba'),
 ('MEA-BST Sox6 Gaba', 'MEA-BST Sox6 Gaba'),
 ('MEA-COA-BMA Ccdc42 Glut', 'MEA-COA-BMA Ccdc42 Glut'),
 ('MH Tac2 Glut', 'MH Tac2 Glut'),
 ('MH Tac2 Glut', 'OPC NN'),
 ('MH Tac2 Glut', 'Oligo NN'),
 ('MM Foxb1 Glut', 'MM Foxb1 Glut'),
 ('MM-ant Foxb1 Glut', 'MM-ant Foxb1 Glut'),
 ('MPN-MPO-PVpo Hmx2 Glut', 'MPN-MPO-PVpo Hmx2 Glut'),
 ('MPO-ADP Lhx8 Gaba', 'MPO-ADP Lhx8 Gaba'),
 ('MRN-VTN-PPN Pax5 Cdh23 Gaba', 'MRN-VTN-PPN Pax5 Cdh23 Gaba'),
 ('MS-SF Bsx Glut', 'MS-SF Bsx Glut'),
 ('MV-SPIV-PRP Dmbx1 Gly-Gaba', 'MV-SPIV-PRP Dmbx1 Gly-Gaba'),
 ('Microglia NN', 'Microglia NN'),
 ('Microglia NN', 'OB Dopa-Gaba'),
 ('Microglia NN', 'OB Meis2 Thsd7b Gaba'),
 ('Microglia NN', 'OB-STR-CTX Inh IMN'),
 ('Microglia NN', 'OB-out Frmd7 Gaba'),
 ('Microglia NN', 'OEC NN'),
 ('Microglia NN', 'OPC NN'),
 ('Microglia NN', 'Oligo NN'),
 ('Microglia NN', 'Peri NN'),
 ('Microglia NN', 'SMC NN'),
 ('Microglia NN', 'VLMC NN'),
 ('ND-INC Foxd2 Glut', 'ND-INC Foxd2 Glut'),
 ('NDB-SI-MA-STRv Lhx8 Gaba', 'NDB-SI-MA-STRv Lhx8 Gaba'),
 ('NDB-SI-ant Prdm12 Gaba', 'NDB-SI-ant Prdm12 Gaba'),
 ('NLL Gata3 Gly-Gaba', 'NLL Gata3 Gly-Gaba'),
 ('NP PPP Glut', 'NP PPP Glut'),
 ('NP SUB Glut', 'NP SUB Glut'),
 ('NTS Dbh Glut', 'NTS Dbh Glut'),
 ('NTS Phox2b Glut', 'NTS Phox2b Glut'),
 ('NTS-PARN Neurod2 Gly-Gaba', 'NTS-PARN Neurod2 Gly-Gaba'),
 ('OB Dopa-Gaba', 'OB Dopa-Gaba'),
 ('OB Dopa-Gaba', 'OB Eomes Ms4a15 Glut'),
 ('OB Dopa-Gaba', 'OB Meis2 Thsd7b Gaba'),
 ('OB Dopa-Gaba', 'OB-STR-CTX Inh IMN'),
 ('OB Dopa-Gaba', 'OB-out Frmd7 Gaba'),
 ('OB Dopa-Gaba', 'Oligo NN'),
 ('OB Dopa-Gaba', 'Peri NN'),
 ('OB Eomes Ms4a15 Glut', 'OB Eomes Ms4a15 Glut'),
 ('OB Eomes Ms4a15 Glut', 'OB Meis2 Thsd7b Gaba'),
 ('OB Eomes Ms4a15 Glut', 'OB-out Frmd7 Gaba'),
 ('OB Meis2 Thsd7b Gaba', 'OB Meis2 Thsd7b Gaba'),
 ('OB Meis2 Thsd7b Gaba', 'OB-STR-CTX Inh IMN'),
 ('OB Meis2 Thsd7b Gaba', 'OB-out Frmd7 Gaba'),
 ('OB Meis2 Thsd7b Gaba', 'OPC NN'),
 ('OB Trdn Gaba', 'OB Trdn Gaba'),
 ('OB Trdn Gaba', 'OB-in Frmd7 Gaba'),
 ('OB-STR-CTX Inh IMN', 'OB-STR-CTX Inh IMN'),
 ('OB-STR-CTX Inh IMN', 'OB-out Frmd7 Gaba'),
 ('OB-STR-CTX Inh IMN', 'OPC NN'),
 ('OB-STR-CTX Inh IMN', 'Oligo NN'),
 ('OB-STR-CTX Inh IMN', 'Peri NN'),
 ('OB-in Frmd7 Gaba', 'OB-in Frmd7 Gaba'),
 ('OB-mi Frmd7 Gaba', 'OB-mi Frmd7 Gaba'),
 ('OB-out Frmd7 Gaba', 'OB-out Frmd7 Gaba'),
 ('OB-out Frmd7 Gaba', 'OPC NN'),
 ('OB-out Frmd7 Gaba', 'Oligo NN'),
 ('OB-out Frmd7 Gaba', 'Peri NN'),
 ('OEC NN', 'OEC NN'),
 ('OEC NN', 'OPC NN'),
 ('OEC NN', 'Oligo NN'),
 ('OEC NN', 'Peri NN'),
 ('OEC NN', 'VLMC NN'),
 ('OPC NN', 'OPC NN'),
 ('OPC NN', 'Oligo NN'),
 ('OPC NN', 'Peri NN'),
 ('OPC NN', 'STR Prox1 Lhx6 Gaba'),
 ('OPC NN', 'Sst Gaba'),
 ('OT D3 Folh1 Gaba', 'OT D3 Folh1 Gaba'),
 ('OT D3 Folh1 Gaba', 'SI-MA-LPO-LHA Skor1 Glut'),
 ('Oligo NN', 'Oligo NN'),
 ('Oligo NN', 'Peri NN'),
 ('Oligo NN', 'RT-ZI Gnb3 Gaba'),
 ('Oligo NN', 'SI-MA-LPO-LHA Skor1 Glut'),
 ('Oligo NN', 'STR-PAL Chst9 Gaba'),
 ('Oligo NN', 'Sst Chodl Gaba'),
 ('Oligo NN', 'TRS-BAC Sln Glut'),
 ('PAG Pou4f1 Ebf2 Glut', 'PAG Pou4f1 Ebf2 Glut'),
 ('PAG Pou4f2 Glut', 'PAG Pou4f2 Glut'),
 ('PAG Pou4f2 Mesi2 Glut', 'PAG Pou4f2 Mesi2 Glut'),
 ('PAG-MRN Pou3f1 Glut', 'PAG-MRN Pou3f1 Glut'),
 ('PAG-MRN-RN Foxa2 Gaba', 'PAG-MRN-RN Foxa2 Gaba'),
 ('PAG-ND-PCG Onecut1 Gaba', 'PAG-ND-PCG Onecut1 Gaba'),
 ('PAG-PPN Pax5 Sox21 Gaba', 'PAG-PPN Pax5 Sox21 Gaba'),
 ('PAG-SC Pou4f1 Zic1 Glut', 'PAG-SC Pou4f1 Zic1 Glut'),
 ('PAL-STR Gaba-Chol', 'PAL-STR Gaba-Chol'),
 ('PARN-MDRNd-NTS Gbx2 Gly-Gaba', 'PARN-MDRNd-NTS Gbx2 Gly-Gaba'),
 ('PAS-MV Ebf2 Gly-Gaba', 'PAS-MV Ebf2 Gly-Gaba'),
 ('PB Evx2 Glut', 'PB Evx2 Glut'),
 ('PB Lmx1a Glut', 'PB Lmx1a Glut'),
 ('PB-NTS Phox2b Ebf3 Lmx1b Glut', 'PB-NTS Phox2b Ebf3 Lmx1b Glut'),
 ('PBG Mtnr1a Glut-Chol', 'PBG Mtnr1a Glut-Chol'),
 ('PCG-PRNr Vsx2 Nkx6-1 Glut', 'PCG-PRNr Vsx2 Nkx6-1 Glut'),
 ('PDTg Otp Olig3 Gaba', 'PDTg Otp Olig3 Gaba'),
 ('PH-LHA Foxb1 Glut', 'PH-LHA Foxb1 Glut'),
 ('PH-SUM Foxa1 Glut', 'PH-SUM Foxa1 Glut'),
 ('PH-ant-LHA Otp Bsx Glut', 'PH-ant-LHA Otp Bsx Glut'),
 ('PMd-LHA Foxb1 Glut', 'PMd-LHA Foxb1 Glut'),
 ('PPN-CUN-PCG Otp En1 Gaba', 'PPN-CUN-PCG Otp En1 Gaba'),
 ('PRNc Otp Gly-Gaba', 'PRNc Otp Gly-Gaba'),
 ('PRNc-NI-SG-RPO Vsx2 Nr4a2 Glut', 'PRNc-NI-SG-RPO Vsx2 Nr4a2 Glut'),
 ('PRP-NI-PRNc-GRN Otp Glut', 'PRP-NI-PRNc-GRN Otp Glut'),
 ('PRT Tcf7l2 Gaba', 'PRT Tcf7l2 Gaba'),
 ('PVH-SO-PVa Otp Glut', 'PVH-SO-PVa Otp Glut'),
 ('PVHd-DMH Lhx6 Gaba', 'PVHd-DMH Lhx6 Gaba'),
 ('PVR Six3 Sox3 Gaba', 'PVR Six3 Sox3 Gaba'),
 ('PVT-PT Ntrk1 Glut', 'PVT-PT Ntrk1 Glut'),
 ('PVpo-VMPO-MPN Hmx2 Gaba', 'PVpo-VMPO-MPN Hmx2 Gaba'),
 ('Peri NN', 'Peri NN'),
 ('Peri NN', 'SMC NN'),
 ('Peri NN', 'VLMC NN'),
 ('Pineal Crx Glut', 'Pineal Crx Glut'),
 ('Pvalb Gaba', 'Pvalb Gaba'),
 ('RHP-COA Ndnf Gaba', 'RHP-COA Ndnf Gaba'),
 ('RPA Pax6 Hoxb5 Gly-Gaba', 'RPA Pax6 Hoxb5 Gly-Gaba'),
 ('RT-ZI Gnb3 Gaba', 'RT-ZI Gnb3 Gaba'),
 ('SBPV-PVa Six6 Satb2 Gaba', 'SBPV-PVa Six6 Satb2 Gaba'),
 ('SC Lef1 Otx2 Gaba', 'SC Lef1 Otx2 Gaba'),
 ('SC Lef1 Otx2 Gaba', 'SCsg Pde5a Glut'),
 ('SC Tnnt1 Gli3 Gaba', 'SC Tnnt1 Gli3 Gaba'),
 ('SCH Six6 Cdc14a Gaba', 'SCH Six6 Cdc14a Gaba'),
 ('SCig Foxb1 Glut', 'SCig Foxb1 Glut'),
 ('SCig Tfap2b Chrnb3 Glut', 'SCig Tfap2b Chrnb3 Glut'),
 ('SCiw Pitx2 Glut', 'SCiw Pitx2 Glut'),
 ('SCop Sln Glut', 'SCop Sln Glut'),
 ('SCs Dmbx1 Gaba', 'SCs Dmbx1 Gaba'),
 ('SCs Pax7 Nfia Gaba', 'SCs Pax7 Nfia Gaba'),
 ('SCsg Gabrr2 Gaba', 'SCsg Gabrr2 Gaba'),
 ('SCsg Gabrr2 Gaba', 'SCsg Pde5a Glut'),
 ('SCsg Pde5a Glut', 'SCsg Pde5a Glut'),
 ('SCsg Pde5a Glut', 'VLMC NN'),
 ('SI-MA-LPO-LHA Skor1 Glut', 'SI-MA-LPO-LHA Skor1 Glut'),
 ('SI-MPO-LPO Lhx8 Gaba', 'SI-MPO-LPO Lhx8 Gaba'),
 ('SMC NN', 'SMC NN'),
 ('SMC NN', 'VLMC NN'),
 ('SNc-VTA-RAmb Foxa1 Dopa', 'SNc-VTA-RAmb Foxa1 Dopa'),
 ('SPA-SPFm-SPFp-POL-PIL-PoT Sp9 Glut', 'SPA-SPFm-SPFp-POL-PIL-PoT Sp9 Glut'),
 ('SPVC Ccdc172 Glut', 'SPVC Ccdc172 Glut'),
 ('SPVI-SPVC Sall3 Lhx1 Gly-Gaba', 'SPVI-SPVC Sall3 Lhx1 Gly-Gaba'),
 ('STN-PSTN Pitx2 Glut', 'STN-PSTN Pitx2 Glut'),
 ('STR D1 Gaba', 'STR D1 Gaba'),
 ('STR D1 Sema5a Gaba', 'STR D1 Sema5a Gaba'),
 ('STR D1 Sema5a Gaba', 'STR-PAL Chst9 Gaba'),
 ('STR D2 Gaba', 'STR D2 Gaba'),
 ('STR Lhx8 Gaba', 'STR Lhx8 Gaba'),
 ('STR Prox1 Lhx6 Gaba', 'STR Prox1 Lhx6 Gaba'),
 ('STR-PAL Chst9 Gaba', 'STR-PAL Chst9 Gaba'),
 ('Sncg Gaba', 'Sncg Gaba'),
 ('Sst Chodl Gaba', 'Sst Chodl Gaba'),
 ('Sst Gaba', 'Sst Gaba'),
 ('TMd-DMH Foxd2 Gaba', 'TMd-DMH Foxd2 Gaba'),
 ('TMv-PMv Tbx3 Hist-Gaba', 'TMv-PMv Tbx3 Hist-Gaba'),
 ('TRS-BAC Sln Glut', 'TRS-BAC Sln Glut'),
 ('TU-ARH Otp Six6 Gaba', 'TU-ARH Otp Six6 Gaba'),
 ('Tanycyte NN', 'Tanycyte NN'),
 ('VLMC NN', 'VLMC NN'),
 ('VMH Nr5a1 Glut', 'VMH Nr5a1 Glut'),
 ('Vip Gaba', 'Vip Gaba'),
 ('ZI Pax6 Gaba', 'ZI Pax6 Gaba')}

In [11]:
len(pair_set - pair_set_15um)

350

In [12]:
len(pair_set.union(pair_set_15um))

707

In [13]:
len([p for p in (pair_set - pair_set_15um) if p[0] == p[1]])

54

In [14]:
len([p for p in pair_set.union(pair_set_15um) if 
  ((p[0].startswith('Astro') and not p[1].endswith('NN'))
  or (p[1].startswith('Astro') and not p[0].endswith('NN'))
  )])

65

In [15]:
len([p for p in pair_set.union(pair_set_15um) if ((not p[0].endswith('NN')) and (not p[1].endswith('NN')))])

349

In [16]:
len(set([tuple(p) for p in all_lr_df[['subclass_ligand', 'subclass_receptor', 'LR_pair']].values]))

32940

In [17]:
all_lr_df

,subclass_ligand,subclass_receptor,LR_pair,N_contact_pairs,mean_contact_lr_exp,exp_fraction,fold_change,pval,pval_adjusted
346032,DCO Il22 Gly-Gaba,CB Granule Glut,SEMA3B_NRP1_PLXNA4,660,0.432596,0.896970,101.846090,0.000000,0.000000
109188,VLMC NN,CHOR NN,WNT2_FZD8_LRP5,1741,0.039052,0.695577,100.942245,0.000000,0.000000
346061,DCO Il22 Gly-Gaba,CB Granule Glut,SEMA3C_NRP1_NRP2,660,0.169779,0.936364,97.998955,0.000000,0.000000
109185,VLMC NN,CHOR NN,WNT2_FZD5_LRP5,1741,0.066515,0.650775,95.667080,0.000000,0.000000
109092,VLMC NN,CHOR NN,GDF10_ACVR1B_ACVR2A,1741,0.200519,0.781735,88.940020,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...
58687,Astroependymal NN,VLMC NN,LAMB1_ITGA1_ITGB1,129,0.289240,0.759690,2.108373,0.000883,0.009906
248562,ABC NN,Endo NN,SEMA3F_NRP2_PLXNA1,220,0.066111,0.581818,3.242334,0.000886,0.009937
54305,Lymphoid NN,VLMC NN,SPP1_ITGA5_ITGB1,171,0.085624,0.725146,2.691603,0.000887,0.009940
279143,SCsg Pde5a Glut,SC Tnnt1 Gli3 Gaba,JAG2_NOTCH4,246,0.018288,0.447154,2.181392,0.000887,0.009945
